In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
# 多行输出
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

# Lesson 1 

## 导入包

导入的时候，使用 

1. from fastai import *
2. from fastai.[application_name](https://docs.fast.ai/applications.html) import *

只在使用 fastai 的使用使用 * 导入全部，其他的包，使用什么导入什么

- 目前 fastai支持的四种applications:
    1. Computer vision
    2. Natural language text
    3. Tabular data
    4. Collaborative filtering

In [2]:
from fastai import *
from fastai.vision import *

## 数据集

使用 [Oxford-IIIT Pet Dataset](http://www.robots.ox.ac.uk/~vgg/data/pets/) by [O. M. Parkhi et al., 2012](http://www.robots.ox.ac.uk/~vgg/publications/2012/parkhi12a/parkhi12a.pdf)
这个数据集包括12个猫的品种和25个狗的品种。我们的任务是对这37个品种进行分类。2012年的分类结果为59.21%。
我们使用 `untar_data` 函数，这个函数使用数据集的下载地址 URL 作为参数，这个函数可以下载和解压数据
（原始数据为tgz压缩包）。其返回数据集存储的路径。
为了保证GPU能够有效的并行计算，通常使用同等大小的图片，如 224 x 224

In [3]:
help(untar_data)

Help on function untar_data in module fastai.datasets:

untar_data(url:str, fname:Union[pathlib.Path, str]=None, dest:Union[pathlib.Path, str]=None, data=True, force_download=False) -> pathlib.Path
    Download `url` to `fname` if it doesn't exist, and un-tgz to folder `dest`.



`Union[pathlib.Path, str]` 表示要么是 pathlib.Path，要么是 str

In [4]:
path = untar_data(URLs.PETS) # 下载数据
path # 查看数据存储路径

WindowsPath('E:/.fastai/data/oxford-iiit-pet')

# 直接下载数据
```bash
$ wget -c https://s3.amazonaws.com/fast-ai-imageclas/oxford-iiit-pet.tgz
```

In [5]:
# 查看路径下的文件
path.ls()

[WindowsPath('E:/.fastai/data/oxford-iiit-pet/annotations'),
 WindowsPath('E:/.fastai/data/oxford-iiit-pet/images')]

In [6]:
path_anno = path/'annotations'  # 标注文件夹
path_img = path/'images'  # 图片存储文件夹

In [7]:
help(get_image_files)

Help on function get_image_files in module fastai.vision.data:

get_image_files(c:Union[pathlib.Path, str], check_ext:bool=True, recurse=False) -> Collection[pathlib.Path]
    Return list of files in `c` that are images. `check_ext` will filter to `image_extensions`.



`get_image_files` 函数获取路径 c 下的所有图片数据，并返回一个图片文件名/路径的列表

In [8]:
fnames = get_image_files(path_img)
fnames[:5]

[WindowsPath('E:/.fastai/data/oxford-iiit-pet/images/Abyssinian_1.jpg'),
 WindowsPath('E:/.fastai/data/oxford-iiit-pet/images/Abyssinian_10.jpg'),
 WindowsPath('E:/.fastai/data/oxford-iiit-pet/images/Abyssinian_100.jpg'),
 WindowsPath('E:/.fastai/data/oxford-iiit-pet/images/Abyssinian_101.jpg'),
 WindowsPath('E:/.fastai/data/oxford-iiit-pet/images/Abyssinian_102.jpg')]

注意这里有个坑，Windows 使用 `\` 作为路径分割符，所以使用 `\\` 代表真正的 `\` (`\`为转义符号)

但这一个问题已经在 1.0.49 中得到修复, 也就是可以使用 `/` 代表路径分隔符和操作系统无关

In [9]:
np.random.seed(2)  # 设置随机状态
pat = r'/([^/]+)_\d+.jpg$'  # 设置正则匹配模式
# pat = r'\\([^\\]+)_\d+.jpg$'  # 设置正则匹配模式 windows 格式
# 上述匹配模式的含义 以/开头，内容不含/的一个或者多个字符紧跟着是_然后是1个或多个数字以.jpg结尾，提取的信息是不含/的字符串
# import re
# x = 'E:/.fastai/data/oxford-iiit-pet/images/Abyssinian_102.jpg'
# match = re.search(pat, x)  # 匹配结果 /Abyssinian_102.jpg
# match.group(1)  # 提取的信息 Abyssinian

- **！坑！** 在 Windows上创建数据集的时候一定要设置 num_workers=0, 否则会报错 `[Errno 32] Broken pipe` 
- 无论使用 PyTorch 或者 fastai 都这样做

In [10]:
bs = 64  # batchsize
# 通过正则匹配来创建数据集
data = ImageDataBunch.from_name_re(path_img, fnames, pat, ds_tfms=get_transforms(), 
#                                    size=224, bs=bs, num_workers=0)  # Linux
data = ImageDataBunch.from_name_re(path_img, fnames, pat, ds_tfms=get_transforms(), 
                                   size=224, bs=bs, num_workers=0)  # Windows
data.normalize(imagenet_stats)

SyntaxError: invalid syntax (<ipython-input-10-da32c67e647a>, line 7)

In [ ]:
data.show_batch(rows=3, figsize=(5,5))

In [ ]:
data.classes  # 数据类
len(data.classes) # 数据类的个数
data.c # 类数

## 训练模型

创建学习器

In [ ]:
# 使用 resnet34
learn = cnn_learner(data, models.resnet34, metrics=error_rate)

In [ ]:
# 查看模型
learn

In [ ]:
help(learn.fit_one_cycle)

- cyc_len: epoch 数
- max_lr:Union[float, Collection[float], slice]=slice(None, 0.003, None) 学习率，默认为 0.003
- moms:Tuple[float, float]=(0.95, 0.85) 动量设置， 默认为 (0.95, 0.85)

In [ ]:
# 使用 one cycle策略进行训练 4 epoch
learn.fit_one_cycle(4)

* 上述结果表明：使用resnet34预训练模型，经过4个epoch的训练，我们可以得到 0.07 左右的误差率，远远优于2012年的56.21%的分类准确率
* 这个时候训练的只是我们添加到最后的全连接层的权重

In [ ]:
# 保存训练的模型
learn.save('stage-1')

## 结果

1. 查看损失最大的样本

In [ ]:
help( ClassificationInterpretation.from_learner)

ClassificationInterpretation: [vision.learner | fastai](https://docs.fast.ai/vision.learner.html#ClassificationInterpretation)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)  # fastai的模型解释器
interp

In [ ]:
# 最大的损失，以及它们所在的索引
losses, idxs = interp.top_losses()
len(data.valid_ds)==len(losses)==len(idxs)

In [ ]:
losses, idxs

In [ ]:
interp.plot_top_losses(9, figsize=(15, 8))

上面的标注依次为： prediction, actual, loss, and probability of predicted class

2. 使用混淆矩阵来评价模型

In [ ]:
interp.plot_confusion_matrix(figsize=(12,12), dpi=60)

- 我们可以看到分布严重倾斜:模型一再犯同样的错误，但很少混淆其他类别。这表明它只是发现很难区分某些特定类别;这是正常的行为。
- 在类别比较多的时候，可视化混淆矩阵就会变得困难，这个时候使用most_confused函数

In [ ]:
# 混乱矩阵中最大的非对角项的降序列表，actual, predicted, number of occurrences
# min_val 表示最小预测错误的次数
interp.most_confused(min_val=2)

## 微调学习率

- 通过对不同的layers使用不同的学习率，一般前面的layers学习率小，表示对权重的调整不大，后面的layer学习率大，表示对权重的调整程度较高。
- 首先，需要learn.unfreeze()，使得各个layer的权重是可调整的

In [ ]:
learn.unfreeze()

* 使用 0.03 的初始学习率对各层的权重进行适应性调整

In [11]:
learn.fit_one_cycle(1)

NameError: name 'learn' is not defined

这里我们会发现在对各层的权重做了调整之后，误差率增大了，也就是说对各层使用同一学习率，一视同仁是不合适的

- 通过[LNCS 8689 - Visualizing and Understanding Convolutional Networks](https://cs.nyu.edu/~fergus/papers/zeilerECCV2014.pdf)来解释为什么，上述训练的结果变差了，前面的layer是学习图像的简单特征，比如角，线等，后面的layer学习的是前面的简单特征的组合，所以一开始我们只训练最后的layer，而不改变前面的layer，这使得我们的模型拥有较低的误差率。但是当我们使用同样的学习率来调整所有layer的权重的时候，反而会使得前面layer的简单特征遭到破坏，因此训练效果也会相应的变差。
* 所以，我们不可以使用上述的训练方式对模型进行训练

In [ ]:
# 加载回原来的微调模型状态
learn.load('stage-1')

- 使用 `lr_find()` 函数，查找最优的学习率
- `recorder.plot()` 会绘制 loss-lr 的图，从中我们可以获取学习率的一个范围

In [ ]:
learn.lr_find()
learn.recorder.plot()

- 学习率可以简单地理解为更新模型中的参数的速度有多快。
- 从上述的图中，可以看出，一旦学习率超过 1e-4 损失就会迅速增加，所以，我们的学习率不应该超过1e-4
- `max_lr=slice(1e-6,1e-4)` 表示在开头的layer使用 1e-6 的学习率，在最后的layer使用 1e-4 的学习率。中间的layer的学习率处于二者之间均匀分布
- 一般 max_lr 的最大学习率是第一阶段（只训练最后layer）的 1/10

In [ ]:
learn.unfreeze()  # 训练所有的层
learn.fit_one_cycle(2, max_lr=slice(1e-6,1e-4))

- 这次的误差率在 0.06 左右，比第一个阶段的要稍微好一些

In [ ]:
# 最终，保存训练好的模型
learn.save('stage-2')

## 使用 resnet50

- 网络层数变多，也意味着模型的复杂度变高，也会有更多的权重需要学习
- 为了使得内存不溢出，我们也要使用更小的 batchsize
- 这里展示完整的学习过程

### step1 创建数据

In [ ]:
data = ImageDataBunch.from_name_re(path_img, fnames, pat, ds_tfms=get_transforms(), 
                                   size=224, bs=bs//2, num_workers=0)  # Windows
# data = ImageDataBunch.from_name_re(path_img, fnames, pat, ds_tfms=get_transforms(), 
#                                    size=224, bs=bs//2)  # Linux
data.normalize(imagenet_stats)

In [ ]:
imagenet_stats  # imagenet 图片的均值、标准差

### step2 创建学习器

In [ ]:
learn = cnn_learner(data, models.resnet50, metrics=error_rate)

### stage 1：对最后的layer进行权重学习

In [ ]:
learn.fit_one_cycle(8)

In [ ]:
learn.save('stage-1-50')

### stage2：对所有layer的权重进行微调

In [ ]:
# 寻找最优unfreezeeezefreezefreeze率
learn.lr_find()
learn.recorder.plot()

In [ ]:
# 微调权重
learn.unfreeze()
learn.fit_one_cycle(3, max_lr=slice(1e-6,1e-4))

In [ ]:
learn.save('stage-2-50')

* 最后我们选择的模型是 stage1 或者 stage2 中的一个，具体取决于模型的效果

### 解释模型

In [ ]:
# 导入模型并分析训练结果
learn.load('stage-2-50')
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.most_confused(min_val=2)

## MNIST 数据集

In [ ]:
# 获取数据
path = untar_dataar_data(URLs.MNIST_SAMPLE)

In [12]:
tfms = get_transforms(do_flip=False)  # 不对图片进行随机翻转，默认情况下会对图片进行随机翻转
# 使用 from_folder 的方式创建数据
data = ImageDataBunch.from_folder(path, ds_tfms=tfms, size=26, num_workers=0)  # 只有训练集验证集的数据，没有标签

C:\Users\liu\AppData\Roaming\Python\Python36\site-packages\fastai\data_block.py:436: UserWarning: Your training set is empty. If this is by design, pass `ignore_empty=True` to remove this warning.
  warn("Your training set is empty. If this is by design, pass `ignore_empty=True` to remove this warning.")
C:\Users\liu\AppData\Roaming\Python\Python36\site-packages\fastai\data_block.py:439: UserWarning: Your validation set is empty. If this is by design, use `split_none()`
                 or pass `ignore_empty=True` when labelling to remove this warning.
  or pass `ignore_empty=True` when labelling to remove this warning.""")


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
data # 此时的CategoryList是具体的类名

In [ ]:
# 查看部分数据
data.show_batch(rows=3, figsize=(5,5))

In [ ]:
# 创建学习器 resnet18，使用 精确度 作为评价标准
learn = cnn_learner(data, models.resnet18, metrics=accuracy)
learn.fit(2)  # stage1 训练 2 epoch

In [ ]:
# 读入标签
df = pd.read_csv(path/'labels.csv')
df.head()

In [ ]:
data = ImageDataBunch.from_csv(path, ds_tfms=tfms, size=28, num_workers=0) # 添加标签的同时一并改变图片的大小
data  # 此时的CategoryList是经过处理的类名

In [13]:
# 再次查看图片
data.show_batch(rows=3, figsize=(5,5))
data.classes  # 类别对应的标注 (0, 1)
data.c

NameError: name 'data' is not defined

In [ ]:
# 获取文件名的列表
fn_paths = [path/name for name in df['name']]
fn_paths[:2]

In [ ]:
pat = r"/(\d)/\d+\.png$"
data = ImageDataBunch.from_name_re(path, fn_paths, pat=pat, ds_tfms=tfms, size=24, num_workers=0)
data.classes  # 类别对应的名称

In [ ]:
# 获取类别名称的另外一种方法
data = ImageDataBunch.from_name_func(path, fn_paths, ds_tfms=tfms, size=24,num_workers=0,
        label_func = lambda x: '3' if '/3/' in str(x) else '7')
data.classes

In [ ]:
# 获取对应的标签
labels = [('3' if '/3/' in str(x) else '7') for x in fn_paths]
labels[:5]

In [ ]:
# 获取类别名称的另外一种方法
data = ImageDataBunch.from_lists(path, fn_paths, labels=labels, ds_tfms=tfms, size=24, num_workers=0)
data.classes

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(8, max_lr=slice(5e-6, 1e-3))